In [26]:
import pandas as pd
import numpy as np

import requests

from datetime import datetime, timedelta
from pprint import pprint

import plotly.express as px
import plotly.colors as pc

### EJEMPLO


In [27]:
lang    =  "es"
category = "balance"
widget   = "balance-electrico"
query   = "start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=year"


In [28]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [29]:
endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
response = requests.get(url = endpoint, headers = headers)
data = response.json()
pprint(data)

{'data': {'attributes': {'description': 'Balance eléctrico: asignación de '
                                        'unidades de producción según '
                                        'combustible principal. La producción '
                                        'neta de las instalaciones no '
                                        'renovables e hidráulicas UGH tiene '
                                        'descontados los consumos propios de '
                                        'las instalaciones. En dichos tipos de '
                                        'producción una generación negativa '
                                        'indica que la electricidad horaria '
                                        'consumida para los usos de las '
                                        'plantas excede la producción horaria '
                                        'de las instalaciones.',
                         'last-update': '2019-06-12T17:40:40.000+02:00',
              

# OBTENCIÓN DE DATOS DE BALANCE




In [30]:
lang    =  input('¿En español (es) o en inglés (en)?') 

restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

In [31]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [32]:
def balance_datos(lang, category, widget):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')


    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"



    endpoint = f"https://apidatos.ree.es/{lang}/datos/balance/balance-electrico?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()



    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()
    lista_dias = list()
    lista_meses = list()
    lista_años = list()


    for dato in data['included']: 
        for info in dato['attributes']['content']:
            nombre = info['type']

            tipo = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)
                dia = fecha.strftime('%d')
                mes = fecha.strftime('%m')
                año = fecha.strftime('%Y')

                fecha = fecha.strftime("%d/%m/%Y")

        
            
            
                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)
                lista_dias.append(dia)
                lista_meses.append(mes)
                lista_años.append(año)

    df_balance = pd.DataFrame()
    df_balance['nombre']               = lista_nombres
    df_balance['tipo de energía']      = lista_tipos
    df_balance['Valores']              = lista_valores
    df_balance["Porcentaje"]           = lista_porcentajes
    df_balance["Fecha actualización"]  = lista_fechas
               




    return df_balance

df_balance  = balance_datos(lang, category, widget)

df_balance['tipo de energía'].unique()


array(['Renovable', 'No-Renovable', 'Demanda en b.c.'], dtype=object)

In [33]:
df_balance.to_csv('balance_electrico.csv')

In [34]:
colores_personalizados = px.colors.qualitative.Plotly + px.colors.qualitative.Pastel + px.colors.qualitative.Set1
colores_personalizados = colores_personalizados[:30]

In [52]:
fig_all = px.line(data_frame = df_balance,
        x = 'Fecha actualización',
        y = 'Valores',
        color = 'nombre',
        color_discrete_sequence = colores_personalizados,
)

fig_all.update_layout(title = 'Evolución de energía diaría unificada')

fig_all.show()


/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [49]:
fig_reno = px.line(data_frame = df_balance[df_balance['tipo de energía'] == 'Renovable'],
        x = 'Fecha actualización',
        y = 'Valores',
        color = 'nombre',
        color_discrete_sequence = colores_personalizados
)

fig_reno.update_layout(title = 'Evolución de energía diaría renovable')

fig_reno.show()

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [50]:
fig_no_reno=px.line(data_frame = df_balance[df_balance['tipo de energía'] == 'No-Renovable'],
        x = 'Fecha actualización',
        y = 'Valores',
        color = 'nombre',
        color_discrete_sequence = colores_personalizados
)

fig_no_reno.update_layout(title = 'Evolución de energía diaría no renovable')

fig_no_reno.show()

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [51]:

fig_dbc = px.line(data_frame = df_balance[df_balance['tipo de energía'] == 'Demanda en b.c.'],
        x = 'Fecha actualización',
        y = 'Valores',
        color = 'nombre',
        color_discrete_sequence = colores_personalizados
)

fig_dbc.update_layout(title = 'Evolución de energía diaría de demanda en barra central')

fig_dbc.show()

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [46]:
px.scatter(data_frame = df_balance,
            y = 'Valores',
            color = 'nombre',
            color_discrete_sequence = colores_personalizados,
            opacity     = 0.3,
            size        = 'Porcentaje',
            size_max    = 30
          )

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [42]:
px.scatter(data_frame = df_balance,
            y = 'Valores',
            color = 'tipo de energía',
            color_discrete_sequence = colores_personalizados,
            opacity     = 0.5,
            hover_name  = 'nombre',
            size        = 'Porcentaje',
            size_max    = 30
          )

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [43]:
px.box(data_frame=df_balance,
       x = 'Valores',
       y = 'nombre',
       color = 'nombre',
       color_discrete_sequence = colores_personalizados)

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [44]:
px.box(data_frame=df_balance,
       x = 'Valores',
       y = 'tipo de energía',
       color = 'tipo de energía',
       color_discrete_sequence = colores_personalizados)

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

